In [7]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

# Load the data
store = pd.read_csv('store.csv')
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv', low_memory=False)

# Merge the DataFrames on the 'Store' column
trainStore = pd.merge(train, store, on='Store')
trainStore.to_csv('trainStore.csv', index=False)
trainStore = trainStore.dropna()

# Encoding categorical variables
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
trainStore_encoded_label = trainStore.copy()

for column in trainStore_encoded_label.columns:
    if trainStore_encoded_label[column].dtype == 'object':
        trainStore_encoded_label[column] = label_encoder.fit_transform(trainStore_encoded_label[column])

# Define the CNN model
class CatsDogsModel(nn.Module):
    def __init__(self, hidden_size):
        super(CatsDogsModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(128 * 28 * 28, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 2)  # Output 2 classes (cats and dogs)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        self.dropout = nn.Dropout(p=0.6)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = x.view(-1, 128 * 28 * 28)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Set hyperparameters
lr = 0.001
batch_size = 20
epochs = 10
hidden_size = 50

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Define dataset class
class CatsDogsDataset(data.Dataset):
    def __init__(self, directory, transform=None, train=True):
        self.directory = directory
        self.filepaths = [os.path.join(directory, file) for file in os.listdir(directory)]
        self.labels = [1 if file.startswith("dog") else 0 for file in os.listdir(directory)]
        self.transform = transform
        if train:
            self.filepaths = self.filepaths[:int(0.8 * len(self.filepaths))]
            self.labels = self.labels[:int(0.8 * len(self.labels))]
        else:
            self.filepaths = self.filepaths[int(0.8 * len(self.filepaths)):]
            self.labels = self.labels[int(0.8 * len(self.labels)):]

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, index):
        img = Image.open(self.filepaths[index]).convert('RGB')
        label = self.labels[index]
        if self.transform:
            img = self.transform(img)
        return img, label

# Define data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = CatsDogsDataset("/Users/raphaelkoop/Developer/catsvsdogs/train", transform=transform, train=True)
test_dataset = CatsDogsDataset("/Users/raphaelkoop/Developer/catsvsdogs/train", transform=transform, train=False)

# Create data loaders
train_loader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize model, optimizer, and loss function
model = CatsDogsModel(hidden_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.0001)
loss_func = nn.CrossEntropyLoss()

# Training loop
train_losses = []
test_losses = []

for epoch in range(epochs):
    model.train()
    epoch_train_losses = []
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}", leave=False):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        
        # Resize labels to match the output batch size
        labels = labels.repeat(outputs.size(0) // labels.size(0))
        
        train_loss = loss_func(outputs, labels)
        train_loss.backward()
        optimizer.step()
        epoch_train_losses.append(train_loss.item())
    train_losses.append(sum(epoch_train_losses) / len(epoch_train_losses))

    model.eval()
    with torch.no_grad():
        epoch_test_losses = []
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            test_loss = loss_func(outputs, labels)
            epoch_test_losses.append(test_loss.item())
    test_losses.append(sum(epoch_test_losses) / len(epoch_test_losses))
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_losses[-1]}, Test Loss: {test_losses[-1]}")

# Plot losses
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Calculate accuracy
correct = 0
total = 0

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print("Accuracy:", accuracy)


Using device: cpu


Epoch 1/10:  64%|██████▍   | 643/1000 [08:45<05:22,  1.11it/s]